In [1]:
!pip -q install huggingface chromadb transformers langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 KB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.3/426.3 KB 18.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 KB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [2]:
!pip install InstructorEmbedding

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from langchain.embeddings import HuggingFaceEmbeddings

In [4]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

Embeddings create a vector representation of a piece of text. This is useful because it means we can think about text in the vector space, and do things like semantic search where we look for pieces of text that are most similar in the vector space.

The base Embedding class in LangChain exposes two methods: embed_documents and embed_query. The largest difference is that these two methods have different interfaces: one works over multiple documents, while the other works over a single document. Besides this, another reason for having these as two separate methods is that some embedding providers have different embedding methods for documents (to be searched over) vs queries (the search query itself).

In [5]:
hfEmbed = HuggingFaceEmbeddings()

In [6]:
hfEmbed.model_name

'sentence-transformers/all-mpnet-base-v2'

In [7]:
hfInstructEmbed = HuggingFaceInstructEmbeddings(
 query_instruction="Represent the query for retrieval: "   
)

load INSTRUCTOR_Transformer
max_seq_length  512


In [8]:
hfInstructEmbed.model_name

'hkunlp/instructor-large'

In [9]:
text = "This is a test document."

In [10]:
query_result = hfEmbed.embed_query(text)

In [ ]:
query_result

In [12]:
doc_result = hfEmbed.embed_documents([text])

In [ ]:
doc_result

In [14]:
instructor_result = hfInstructEmbed.embed_query(text)

In [ ]:
instructor_result

In [16]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader

In [31]:
spaceLoad = TextLoader('/content/linux_play.txt')

In [18]:
from langchain.text_splitter import CharacterTextSplitter

In [32]:
documents = spaceLoad.load()

In [29]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

hf_splitter = CharacterTextSplitter.from_huggingface_tokenizer(tokenizer, 
                                                                  chunk_size=100,
                                                                  chunk_overlap=0)

In [33]:
texts = hf_splitter.split_documents(documents)

In [34]:
from langchain.vectorstores import Chroma

data_space = Chroma.from_documents(texts,hfEmbed)

In [35]:
#using data_space as retriever

space_retriever = data_space.as_retriever()

In [ ]:
#I am betting on using the recent dolly

In [36]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

In [37]:
!pip install configparser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
import configparser

api_reader = configparser.ConfigParser()
api_reader.read_file(open('/content/apidata.config'))

In [39]:
import os

os.environ['HUGGINGFACEHUB_API_TOKEN'] = api_reader["OPENAI"]["HFREAD"]

In [51]:
llm=HuggingFaceHub(repo_id="gpt2-xl",
                   model_kwargs={"temperature":0.1, 
                                 "max_length":490})

In [53]:
llm("There is a ")

KeyboardInterrupt: ignored

In [54]:
qa = RetrievalQA.from_chain_type(llm=llm, 
                                 chain_type="map_reduce", 
                                 retriever=space_retriever)

In [55]:
qa.run("How many different tasks are there?")

ValueError: ignored